<h2>Big Data - Ex3</h2>
<h2>Avlas Kfir</h2>

<h3>Part1 - Term frequency–inverse document frequency (tf-idf)</h3>

BBC Datasets

News article datasets, originating from BBC News
* Consists of 2225 documents from the BBC news website corresponding to stories in five topical areas from 2004-2005.
* Class Labels: 5 (business, entertainment, politics, sport, tech)

In [2]:
import pandas as pd
import math
import collections
from collections import Counter
import operator
import random
from textblob import TextBlob

pd.set_option("display.max_colwidth", 1000)

In [3]:
# Read data

path = "/home/spark-vm/PycharmProjects/BigDataCource/"
folder_name = "BBC_News"

In [4]:
# Read text files

def read_directory(p_category: str):
    return sc.wholeTextFiles(path + "/" + folder_name + "/" + p_category)

In [5]:
# Since the dataset is divided into folders, 
# Set the file name with folder prefix in order to get uniqueness

def set_file_name(t):
    file_name = t[0]
    text = t[1]
    
    new_file_name = file_name[file_name.index(folder_name) + len(folder_name) + 1:]
    new_file_name = new_file_name.replace("/", "_")
    
    return (new_file_name, text)

In [6]:
def get_data():
    d1 = read_directory("bsns").map(set_file_name)
    d2 = read_directory("entrtmnt").map(set_file_name)
    d3 = read_directory("pltcs").map(set_file_name)
    d4 = read_directory("sprt").map(set_file_name)
    d5 = read_directory("tech").map(set_file_name)
    
    return d1.union(d2).union(d3).union(d4).union(d5)

In [7]:
main_data = get_data()

In [8]:
main_data.take(1)

[('bsns_382.txt',
  'Ban on forced retirement under 65\n\nEmployers will no longer be able to force workers to retire before 65, unless they can justify it.\n\nThe government has announced that firms will be barred from 2006 from imposing arbitrary retirement ages. Under new European age discrimination rules, a default retirement age of 65 will be introduced. Workers will be permitted to request staying on beyond this compulsory retirement age, although employers will have the right to refuse. Trade and Industry Secretary Patricia Hewitt said people would not be forced to work longer than they wanted, saying the default age was not a statutory, compulsory retirement age. She said employers would be free to continue employing people for as long as they were competent.\n\nUnder age discrimination proposals from the Department of Trade and Industry last year workers were to be allowed to work on till 70 if they wished.\n\nBusiness leaders had opposed the plan as they said it would be too 

In [9]:
num_of_documents = main_data.count()

In [10]:
num_of_documents

2225

<h3>Document vector representation</h3>

Break text into individual linguistic units

Clean and arrange :
* Lower case
* Remove English stop words
* Remove English possession ('s)
* Remove symbols and numbers
* Remove empty strings
* Remove one letter words
* Singularize words
* Sort by words

In [11]:
def text_to_vec(t):
    
    file_name = t[0]
    text = t[1]
    words= [] 
    
    for txt in text.split("\n"):       # Text to sentences
        for word in txt.split(' '):    # Sentence to words
            words.append(word.lower()) # Word to lower case
    
    return (file_name, words)

In [12]:
data = main_data.map(text_to_vec)

In [13]:
data.take(1)

[('bsns_382.txt',
  ['ban',
   'on',
   'forced',
   'retirement',
   'under',
   '65',
   '',
   'employers',
   'will',
   'no',
   'longer',
   'be',
   'able',
   'to',
   'force',
   'workers',
   'to',
   'retire',
   'before',
   '65,',
   'unless',
   'they',
   'can',
   'justify',
   'it.',
   '',
   'the',
   'government',
   'has',
   'announced',
   'that',
   'firms',
   'will',
   'be',
   'barred',
   'from',
   '2006',
   'from',
   'imposing',
   'arbitrary',
   'retirement',
   'ages.',
   'under',
   'new',
   'european',
   'age',
   'discrimination',
   'rules,',
   'a',
   'default',
   'retirement',
   'age',
   'of',
   '65',
   'will',
   'be',
   'introduced.',
   'workers',
   'will',
   'be',
   'permitted',
   'to',
   'request',
   'staying',
   'on',
   'beyond',
   'this',
   'compulsory',
   'retirement',
   'age,',
   'although',
   'employers',
   'will',
   'have',
   'the',
   'right',
   'to',
   'refuse.',
   'trade',
   'and',
   'industry',
   

In [14]:
# Broadcast stop words to all workers in order to improve efficiency

from stop_words import get_stop_words

stop_words = get_stop_words('en')

broadcast_sw = sc.broadcast(stop_words)

In [15]:
broadcast_sw.value

['a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 "can't",
 'cannot',
 'could',
 "couldn't",
 'did',
 "didn't",
 'do',
 'does',
 "doesn't",
 'doing',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 "hadn't",
 'has',
 "hasn't",
 'have',
 "haven't",
 'having',
 'he',
 "he'd",
 "he'll",
 "he's",
 'her',
 'here',
 "here's",
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 "how's",
 'i',
 "i'd",
 "i'll",
 "i'm",
 "i've",
 'if',
 'in',
 'into',
 'is',
 "isn't",
 'it',
 "it's",
 'its',
 'itself',
 "let's",
 'me',
 'more',
 'most',
 "mustn't",
 'my',
 'myself',
 'no',
 'nor',
 'not',
 'of',
 'off',
 'on',
 'once',
 'only',
 'or',
 'other',
 'ought',
 'our',
 'ours',
 'ourselves',
 'out',
 'over',
 'own',
 'same',
 "shan't",
 'she',
 "she'd",
 "she'll",
 "she's",
 'should',
 "s

In [16]:
def remove_stop_words(d):
    
    file_name = d[0]
    words = d[1]
    w = []
    
    for word in words:
        if word not in broadcast_sw.value:
            w.append(word)
    
    return (file_name , w)

In [17]:
data1 = data.map(remove_stop_words)

In [18]:
data1.take(1)

[('bsns_382.txt',
  ['ban',
   'forced',
   'retirement',
   '65',
   '',
   'employers',
   'will',
   'longer',
   'able',
   'force',
   'workers',
   'retire',
   '65,',
   'unless',
   'can',
   'justify',
   'it.',
   '',
   'government',
   'announced',
   'firms',
   'will',
   'barred',
   '2006',
   'imposing',
   'arbitrary',
   'retirement',
   'ages.',
   'new',
   'european',
   'age',
   'discrimination',
   'rules,',
   'default',
   'retirement',
   'age',
   '65',
   'will',
   'introduced.',
   'workers',
   'will',
   'permitted',
   'request',
   'staying',
   'beyond',
   'compulsory',
   'retirement',
   'age,',
   'although',
   'employers',
   'will',
   'right',
   'refuse.',
   'trade',
   'industry',
   'secretary',
   'patricia',
   'hewitt',
   'said',
   'people',
   'forced',
   'work',
   'longer',
   'wanted,',
   'saying',
   'default',
   'age',
   'statutory,',
   'compulsory',
   'retirement',
   'age.',
   'said',
   'employers',
   'free',
   'co

In [19]:
import re 

def remove_symbols(t):
    file_name = t[0]
    words = t[1]
    w = []
    
    for i in range(len(words)):
        words[i] = words[i].replace("'s", "")  # Remove english possession ('s)
        word = re.sub("[^a-zA-Z\s]", ' ', words[i]).replace(' ','')
        if word and len(word)>2:
            w.append(word)
    
    return (file_name , w)

In [20]:
data2 = data1.map(remove_symbols)

In [21]:
def singularize(t):
    file_name = t[0]
    words = t[1]
    
    for i in range(len(words)):
        blob = TextBlob(words[i])
        singulars = [bw.singularize() for bw in blob.words]
        if len(singulars) > 0:
            words[i] = str(singulars[0])
    
    return (file_name, words)

In [22]:
data3 = data2.map(singularize)

In [23]:
documents = data3

In [24]:
documents.take(1)

[('bsns_382.txt',
  ['ban',
   'forced',
   'retirement',
   'employer',
   'will',
   'longer',
   'able',
   'force',
   'worker',
   'retire',
   'unles',
   'can',
   'justify',
   'government',
   'announced',
   'firm',
   'will',
   'barred',
   'imposing',
   'arbitrary',
   'retirement',
   'age',
   'new',
   'european',
   'age',
   'discrimination',
   'rule',
   'default',
   'retirement',
   'age',
   'will',
   'introduced',
   'worker',
   'will',
   'permitted',
   'request',
   'staying',
   'beyond',
   'compulsory',
   'retirement',
   'age',
   'although',
   'employer',
   'will',
   'right',
   'refuse',
   'trade',
   'industry',
   'secretary',
   'patricium',
   'hewitt',
   'said',
   'person',
   'forced',
   'work',
   'longer',
   'wanted',
   'saying',
   'default',
   'age',
   'statutory',
   'compulsory',
   'retirement',
   'age',
   'said',
   'employer',
   'free',
   'continue',
   'employing',
   'person',
   'long',
   'competent',
   'age',
   '

Building an inverted index

To gain the speed benefits of indexing at retrieval time, we have to build the index in advance.
Index the documents that each term occurs in by creating an inverted index, consisting of a dictionary and postings. 


In [25]:
def invert(d):
    
    doc   = d[0]    
    terms = d[1] 
    
    return [(t,[doc]) for t in terms]

In [26]:
# Invert from doc with [terms] to term with [docs]

terms = documents.flatMap(invert).reduceByKey(lambda x,y: x+y)

In [28]:
terms.take(1)

[('retirement',
  ['bsns_382.txt',
   'bsns_382.txt',
   'bsns_382.txt',
   'bsns_382.txt',
   'bsns_382.txt',
   'bsns_382.txt',
   'bsns_238.txt',
   'bsns_326.txt',
   'bsns_326.txt',
   'bsns_249.txt',
   'bsns_249.txt',
   'bsns_368.txt',
   'bsns_449.txt',
   'bsns_090.txt',
   'bsns_133.txt',
   'bsns_255.txt',
   'bsns_255.txt',
   'bsns_255.txt',
   'bsns_255.txt',
   'bsns_041.txt',
   'bsns_041.txt',
   'bsns_452.txt',
   'entrtmnt_048.txt',
   'entrtmnt_177.txt',
   'entrtmnt_044.txt',
   'entrtmnt_269.txt',
   'entrtmnt_269.txt',
   'pltcs_380.txt',
   'pltcs_042.txt',
   'pltcs_042.txt',
   'pltcs_132.txt',
   'pltcs_132.txt',
   'pltcs_132.txt',
   'pltcs_132.txt',
   'pltcs_132.txt',
   'pltcs_357.txt',
   'pltcs_406.txt',
   'pltcs_294.txt',
   'pltcs_089.txt',
   'pltcs_089.txt',
   'pltcs_156.txt',
   'pltcs_159.txt',
   'pltcs_317.txt',
   'sprt_437.txt',
   'sprt_241.txt',
   'sprt_462.txt',
   'sprt_347.txt',
   'sprt_347.txt',
   'sprt_413.txt',
   'sprt_367.txt'

In [29]:
# Build an Inverted Index structure:

# Posting_list = Dict{doc_name:tf of term in doc}

# Inverted Index = tuple(term,  tuple(df of term in corpus, posting_list) 

def inverted_index(t):
    
    term = t[0]
    docs = dict(Counter(t[1]))
                
    dft = len(docs)
                
    return (term, (dft, docs))

In [30]:
inverted_index = terms.map(inverted_index)

In [31]:
inverted_index.take(1)

[('retirement',
  (38,
   {'bsns_382.txt': 6,
    'bsns_238.txt': 1,
    'bsns_326.txt': 2,
    'bsns_249.txt': 2,
    'bsns_368.txt': 1,
    'bsns_449.txt': 1,
    'bsns_090.txt': 1,
    'bsns_133.txt': 1,
    'bsns_255.txt': 4,
    'bsns_041.txt': 2,
    'bsns_452.txt': 1,
    'entrtmnt_048.txt': 1,
    'entrtmnt_177.txt': 1,
    'entrtmnt_044.txt': 1,
    'entrtmnt_269.txt': 2,
    'pltcs_380.txt': 1,
    'pltcs_042.txt': 2,
    'pltcs_132.txt': 5,
    'pltcs_357.txt': 1,
    'pltcs_406.txt': 1,
    'pltcs_294.txt': 1,
    'pltcs_089.txt': 2,
    'pltcs_156.txt': 1,
    'pltcs_159.txt': 1,
    'pltcs_317.txt': 1,
    'sprt_437.txt': 1,
    'sprt_241.txt': 1,
    'sprt_462.txt': 1,
    'sprt_347.txt': 2,
    'sprt_413.txt': 1,
    'sprt_367.txt': 1,
    'sprt_431.txt': 1,
    'sprt_361.txt': 1,
    'sprt_115.txt': 1,
    'sprt_062.txt': 1,
    'sprt_375.txt': 1,
    'sprt_415.txt': 1,
    'sprt_460.txt': 2}))]

In [32]:
inverted_indexMap = inverted_index.collectAsMap()

In [33]:
inverted_indexMap["retirement"]

(38,
 {'bsns_382.txt': 6,
  'bsns_238.txt': 1,
  'bsns_326.txt': 2,
  'bsns_249.txt': 2,
  'bsns_368.txt': 1,
  'bsns_449.txt': 1,
  'bsns_090.txt': 1,
  'bsns_133.txt': 1,
  'bsns_255.txt': 4,
  'bsns_041.txt': 2,
  'bsns_452.txt': 1,
  'entrtmnt_048.txt': 1,
  'entrtmnt_177.txt': 1,
  'entrtmnt_044.txt': 1,
  'entrtmnt_269.txt': 2,
  'pltcs_380.txt': 1,
  'pltcs_042.txt': 2,
  'pltcs_132.txt': 5,
  'pltcs_357.txt': 1,
  'pltcs_406.txt': 1,
  'pltcs_294.txt': 1,
  'pltcs_089.txt': 2,
  'pltcs_156.txt': 1,
  'pltcs_159.txt': 1,
  'pltcs_317.txt': 1,
  'sprt_437.txt': 1,
  'sprt_241.txt': 1,
  'sprt_462.txt': 1,
  'sprt_347.txt': 2,
  'sprt_413.txt': 1,
  'sprt_367.txt': 1,
  'sprt_431.txt': 1,
  'sprt_361.txt': 1,
  'sprt_115.txt': 1,
  'sprt_062.txt': 1,
  'sprt_375.txt': 1,
  'sprt_415.txt': 1,
  'sprt_460.txt': 2})

In [34]:
vocabulary = list(inverted_indexMap.keys())

<h3>Inverted index</h3>

In [38]:
inv_ind = inverted_index.map(lambda t:(t[0], list(t[1][1].keys())))

In [39]:
cols = ["Term", "Docs"]
inv_ind = inv_ind.toDF(cols)
inv_ind.toPandas()

,Term,Docs
0,retirement,"[bsns_382.txt, bsns_238.txt, bsns_326.txt, bsns_249.txt, bsns_368.txt, bsns_449.txt, bsns_090.txt, bsns_133.txt, bsns_255.txt, bsns_041.txt, bsns_452.txt, entrtmnt_048.txt, entrtmnt_177.txt, entrtmnt_044.txt, entrtmnt_269.txt, pltcs_380.txt, pltcs_042.txt, pltcs_132.txt, pltcs_357.txt, pltcs_406.txt, pltcs_294.txt, pltcs_089.txt, pltcs_156.txt, pltcs_159.txt, pltcs_317.txt, sprt_437.txt, sprt_241.txt, sprt_462.txt, sprt_347.txt, sprt_413.txt, sprt_367.txt, sprt_431.txt, sprt_361.txt, sprt_115.txt, sprt_062.txt, sprt_375.txt, sprt_415.txt, sprt_460.txt]"
1,retire,"[bsns_382.txt, bsns_238.txt, bsns_041.txt, bsns_227.txt, entrtmnt_016.txt, entrtmnt_117.txt, pltcs_045.txt, sprt_057.txt, sprt_484.txt, sprt_185.txt, sprt_413.txt, sprt_367.txt, sprt_028.txt, sprt_500.txt, sprt_031.txt, sprt_144.txt, sprt_068.txt, sprt_459.txt, sprt_156.txt, sprt_480.txt, sprt_421.txt]"
2,government,"[bsns_382.txt, bsns_272.txt, bsns_002.txt, bsns_491.txt, bsns_110.txt, bsns_246.txt, bsns_322.txt, bsns_245.txt, bsns_433.txt, bsns_437.txt, bsns_339.txt, bsns_016.txt, bsns_195.txt, bsns_411.txt, bsns_303.txt, bsns_409.txt, bsns_248.txt, bsns_286.txt, bsns_023.txt, bsns_478.txt, bsns_241.txt, bsns_200.txt, bsns_187.txt, bsns_428.txt, bsns_083.txt, bsns_178.txt, bsns_104.txt, bsns_328.txt, bsns_308.txt, bsns_022.txt, bsns_489.txt, bsns_266.txt, bsns_345.txt, bsns_347.txt, bsns_501.txt, bsns_045.txt, bsns_161.txt, bsns_493.txt, bsns_277.txt, bsns_502.txt, bsns_388.txt, bsns_096.txt, bsns_093.txt, bsns_029.txt, bsns_498.txt, bsns_326.txt, bsns_487.txt, bsns_432.txt, bsns_380.txt, bsns_139.txt, bsns_146.txt, bsns_260.txt, bsns_199.txt, bsns_048.txt, bsns_163.txt, bsns_164.txt, bsns_445.txt, bsns_426.txt, bsns_344.txt, bsns_072.txt, bsns_183.txt, bsns_273.txt, bsns_341.txt, bsns_497.txt, bsns_025.txt, bsns_033.txt, bsns_231.txt, bsns_340.txt, bsns_120.txt, bsns_174.txt, bsns_389.txt, b..."
3,announced,"[bsns_382.txt, bsns_036.txt, bsns_433.txt, bsns_437.txt, bsns_217.txt, bsns_187.txt, bsns_425.txt, bsns_502.txt, bsns_096.txt, bsns_487.txt, bsns_084.txt, bsns_149.txt, bsns_040.txt, bsns_122.txt, bsns_201.txt, bsns_445.txt, bsns_404.txt, bsns_497.txt, bsns_120.txt, bsns_325.txt, bsns_335.txt, bsns_216.txt, bsns_414.txt, bsns_424.txt, bsns_422.txt, bsns_278.txt, bsns_304.txt, bsns_211.txt, bsns_504.txt, bsns_499.txt, bsns_314.txt, bsns_090.txt, bsns_094.txt, bsns_473.txt, bsns_455.txt, bsns_044.txt, bsns_014.txt, bsns_467.txt, bsns_103.txt, bsns_307.txt, bsns_221.txt, bsns_130.txt, bsns_068.txt, bsns_011.txt, bsns_051.txt, bsns_218.txt, bsns_049.txt, bsns_111.txt, bsns_393.txt, bsns_109.txt, bsns_206.txt, bsns_198.txt, bsns_263.txt, bsns_021.txt, bsns_223.txt, bsns_054.txt, bsns_159.txt, bsns_254.txt, bsns_460.txt, entrtmnt_272.txt, entrtmnt_246.txt, entrtmnt_027.txt, entrtmnt_362.txt, entrtmnt_143.txt, entrtmnt_187.txt, entrtmnt_185.txt, entrtmnt_034.txt, entrtmnt_367.txt, entrtmn..."
4,imposing,"[bsns_382.txt, bsns_036.txt, bsns_314.txt, pltcs_290.txt, pltcs_334.txt, sprt_023.txt, sprt_048.txt, tech_275.txt]"
...,...,...
26741,willpower,[tech_401.txt]
26742,hr,[tech_401.txt]
26743,siliconbased,[tech_329.txt]
26744,citizenbased,[tech_010.txt]


In [40]:
def tf(d):
    
    doc   = d[0]
    terms = Counter(d[1])
   
    return(doc, list(terms.keys()), list(terms.values()))

In [41]:
tf = documents.map(tf)

<h3>Term frequency</h3>

The number of times a term occurs in a document

In [42]:
cols = ["Doc", "Terms", "TF(t,d)"]
tf.toDF(cols).toPandas()

,Doc,Terms,"TF(t,d)"
0,bsns_382.txt,"[ban, forced, retirement, employer, will, longer, able, force, worker, retire, unles, can, justify, government, announced, firm, barred, imposing, arbitrary, age, new, european, discrimination, rule, default, introduced, permitted, request, staying, beyond, compulsory, although, right, refuse, trade, industry, secretary, patricium, hewitt, said, person, work, wanted, saying, statutory, free, continue, employing, long, competent, proposal, department, last, year, allowed, till, wished, busines, leader, opposed, plan, costly, cumbersome, british, chamber, commerce, welcomed, latest, thi, move, today, best, world, ability, define, end, point, employeremployee, relationship, employee, flexibility, past, concern, cowardly, complete, uturn, make, mockery, socalled, commitment, outlawing, ageism, leaving, incoming, law, unravel, gordon, lishman, director, general, ...]","[1, 2, 6, 4, 7, 2, 1, 1, 4, 1, 1, 2, 1, 3, 1, 1, 1, 2, 1, 13, 1, 2, 3, 1, 2, 1, 1, 2, 1, 1, 2, 1, 2, 1, 2, 2, 1, 1, 1, 7, 3, 3, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, ...]"
1,bsns_050.txt,"[bad, weather, hit, nestle, sale, combination, rising, raw, material, cost, sluggish, european, economy, swiss, food, drink, giant, revenue, dipped, franc, ice, cream, mineral, water, dampened, wet, summer, however, profit, margin, helped, strong, performance, america, china, raise, dividend, paying, back, debt, said, strength, dollar, disposal, business, challenging, trading, condition, europe, dented, poor, acros, continent, contrast, prolonged, heat, wave, severely, affected, demand, bottled, also, fell, although, chocolate, coffee, frozen, good, petcare, product, performed, better, elsewhere, enjoyed, exceptional, year, north, outperforming, market, term, growth, added, strongly, africa, asium, despite, impact, high, oil, price, political, instability, total, earning, interest, remained, broadly, flat, past, company, ...]","[2, 2, 2, 12, 6, 1, 1, 1, 1, 1, 1, 1, 1, 4, 1, 1, 1, 1, 1, 3, 2, 2, 2, 3, 1, 1, 2, 1, 2, 2, 1, 1, 1, 2, 1, 1, 2, 1, 2, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, ...]"
2,bsns_057.txt,"[economy, tronger, forecast, probably, grew, faster, rate, third, quarter, reported, according, bank, england, deputy, governor, rachel, lomax, private, sector, busines, survey, suggest, stronger, official, estimate, said, collectively, show, rapid, slowdown, house, price, growth, pointed, out, mean, despite, strong, economic, base, will, stay, hold, datum, come, office, national, statistic, on, though, reliable, take, longer, publish, now, boe, calling, delivery, can, make, effective, policy, decision, recent, work, shown, add, value, even, preliminary, available, speech, north, wale, club, due, second, friday, the, mpc, judge, overall, little, higher, currently, indicate, successful, monetary, depend, good, information, cited, late, example, time, weak, figure, published, substantially, revised, ...]","[4, 1, 1, 2, 1, 2, 4, 3, 3, 1, 1, 4, 2, 1, 1, 2, 7, 2, 2, 2, 3, 1, 1, 3, 3, 8, 1, 1, 1, 3, 2, 3, 4, 1, 1, 1, 1, 1, 5, 1, 2, 1, 1, 5, 2, 1, 2, 2, 5, 1, 1, 1, 1, 2, 1, 1, 1, 2, 2, 2, 1, 4, 3, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 3, 5, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...]"
3,bsns_272.txt,"[market, signal, brazilian, recovery, stock, risen, record, high, investor, display, growing, confidence, durability, country, economic, main, bovespa, index, sao, paolo, exchange, closed, point, friday, topping, previou, close, reached, day, buoyancy, reflect, optimism, economy, grow, much, brazil, recovering, last, year, recession, worst, decade, output, declined, president, luiz, inacio, lula, silva, elected, first, workingclas, strongly, criticised, p

In [43]:
def log_tf(d):
    
    doc   = d[0]
    terms = d[1]
    tf    = d[2]
    
    return(doc, terms, [1 + math.log(t, 10) for t in tf])

In [44]:
log_tf = tf.map(log_tf)

<h3>Term frequency - Log normalization</h3>

In [45]:
cols = ["Doc", "Terms", "Log-TF(t,d)"]
log_tf.toDF(cols).toPandas()

,Doc,Terms,"Log-TF(t,d)"
0,bsns_382.txt,"[ban, forced, retirement, employer, will, longer, able, force, worker, retire, unles, can, justify, government, announced, firm, barred, imposing, arbitrary, age, new, european, discrimination, rule, default, introduced, permitted, request, staying, beyond, compulsory, although, right, refuse, trade, industry, secretary, patricium, hewitt, said, person, work, wanted, saying, statutory, free, continue, employing, long, competent, proposal, department, last, year, allowed, till, wished, busines, leader, opposed, plan, costly, cumbersome, british, chamber, commerce, welcomed, latest, thi, move, today, best, world, ability, define, end, point, employeremployee, relationship, employee, flexibility, past, concern, cowardly, complete, uturn, make, mockery, socalled, commitment, outlawing, ageism, leaving, incoming, law, unravel, gordon, lishman, director, general, ...]","[1.0, 1.3010299956639813, 1.7781512503836434, 1.6020599913279623, 1.8450980400142567, 1.3010299956639813, 1.0, 1.0, 1.6020599913279623, 1.0, 1.0, 1.3010299956639813, 1.0, 1.4771212547196624, 1.0, 1.0, 1.0, 1.3010299956639813, 1.0, 2.113943352306837, 1.0, 1.3010299956639813, 1.4771212547196624, 1.0, 1.3010299956639813, 1.0, 1.0, 1.3010299956639813, 1.0, 1.0, 1.3010299956639813, 1.0, 1.3010299956639813, 1.0, 1.3010299956639813, 1.3010299956639813, 1.0, 1.0, 1.0, 1.8450980400142567, 1.4771212547196624, 1.4771212547196624, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.3010299956639813, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.3010299956639813, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.3010299956639813, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.3010299956639813, 1.0, 1.0, 1.0, 1.0, 1.0, ...]"
1,bsns_050.txt,"[bad, weather, hit, nestle, sale, combination, rising, raw, material, cost, sluggish, european, economy, swiss, food, drink, giant, revenue, dipped, franc, ice, cream, mineral, water, dampened, wet, summer, however, profit, margin, helped, strong, performance, america, china, raise, dividend, paying, back, debt, said, strength, dollar, disposal, business, challenging, trading, condition, europe, dented, poor, acros, continent, contrast, prolonged, heat, wave, severely, affected, demand, bottled, also, fell, although, chocolate, coffee, frozen, good, petcare, product, performed, better, elsewhere, enjoyed, exceptional, year, north, outperforming, market, term, growth, added, strongly, africa, asium, despite, impact, high, oil, price, political, instability, total, earning, interest, remained, broadly, flat, past, company, ...]","[1.3010299956639813, 1.3010299956639813, 1.3010299956639813, 2.0791812460476247, 1.7781512503836434, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.6020599913279623, 1.0, 1.0, 1.0, 1.0, 1.0, 1.4771212547196624, 1.3010299956639813, 1.3010299956639813, 1.3010299956639813, 1.4771212547196624, 1.0, 1.0, 1.3010299956639813, 1.0, 1.3010299956639813, 1.3010299956639813, 1.0, 1.0, 1.0, 1.3010299956639813, 1.0, 1.0, 1.3010299956639813, 1.0, 1.3010299956639813, 1.0, 1.3010299956639813, 1.0, 1.0, 1.0, 1.0, 1.0, 1.3010299956639813, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.3010299956639813, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.3010299956639813, 1.0, 1.0, 1.0, 1.0, 1.3010299956639813, 1.0, 1.0, 1.0, 1.0, 1.3010299956639813, 1.0, 1.0, 1.0, 1.0, 1.3010299956639813, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.3010299956639813, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...]"
2,bsns_057.txt,"[economy, tronger, forecast, probably, grew, faster, rate, third, quarter, reported, according, bank, england, deputy, governor, rachel, lomax, private, sector, busines, survey, suggest, stronger, official, estimate, said, collectively, show, rapid, slowdown, house, price, growth, pointed, out, mean, despite, strong, economic, base, will, stay, hold, datum, come, office, national, statistic, on, though, reliable, take, longer, publish, now, bo

In [46]:
def df(t):
    
    term = t[0]
    dft  = t[1][0]
   
    return (term, dft)

In [47]:
df = inverted_index.map(df)

In [48]:
df.take(1)

[('retirement', 38)]

In [49]:
def idf(t):
    
    term = t[0]
    dft  = t[1]
    
    a = num_of_documents / dft
    idft = math.log(a, 10)
    
    return(term, idft)

In [50]:
idf = df.map(idf)

In [51]:
idf.take(1)

[('retirement', 1.7675464187001402)]

In [52]:
idfMap = idf.collectAsMap()

In [53]:
idft = df.join(idf)

idft = idft.map(lambda t:(t[0], t[1][0], t[1][1]))

<h3>Inverse document frequency</h3>

A measure of how much information the word provides, i.e., if it's common or rare across all documents

In [54]:
cols = ["Term", "DF(t)", "IDF(t)"]
idft.toDF(cols).toPandas()

,Term,DF(t),IDF(t)
0,retirement,38,1.767546
1,default,12,2.268149
2,introduced,67,1.521255
3,beyond,83,1.428252
4,right,406,0.738804
...,...,...,...
26741,marginalised,1,3.347330
26742,sorround,1,3.347330
26743,sanity,1,3.347330
26744,succesfull,1,3.347330


In [55]:
def tf_idf(d):
    
    doc   = d[0]
    terms = d[1]
    log_tf    = d[2]
    wtd = []
    
    for i in range(len(terms)):
        wtd.append(log_tf[i] * idfMap[terms[i]])
    
    return (doc, terms, wtd)

In [56]:
wtd = log_tf.map(tf_idf)

<h3>Term frequency–Inverse document frequency</h3>

It is the logarithmically scaled inverse fraction of the documents that contain the word (obtained by dividing the total number of documents by the number of documents containing the term, and then taking the logarithm of that quotient)


A high weight in tf–idf is reached by a high term frequency (in the given document) and a low document frequency of the term in the whole collection of documents; the weights hence tend to filter out common terms. Since the ratio inside the idf's log function is always greater than or equal to 1, the value of idf (and tf–idf) is greater than or equal to 0. As a term appears in more documents, the ratio inside the logarithm approaches 1, bringing the idf and tf–idf closer to 0. 

In [57]:
cols = ["Doc", "Terms", "W(td)= td-idf"]
wtd.toDF(cols).toPandas()

,Doc,Terms,W(td)= td-idf
0,bsns_382.txt,"[ban, forced, retirement, employer, will, longer, able, force, worker, retire, unles, can, justify, government, announced, firm, barred, imposing, arbitrary, age, new, european, discrimination, rule, default, introduced, permitted, request, staying, beyond, compulsory, although, right, refuse, trade, industry, secretary, patricium, hewitt, said, person, work, wanted, saying, statutory, free, continue, employing, long, competent, proposal, department, last, year, allowed, till, wished, busines, leader, opposed, plan, costly, cumbersome, british, chamber, commerce, welcomed, latest, thi, move, today, best, world, ability, define, end, point, employeremployee, relationship, employee, flexibility, past, concern, cowardly, complete, uturn, make, mockery, socalled, commitment, outlawing, ageism, leaving, incoming, law, unravel, gordon, lishman, director, general, ...]","[1.466516423036159, 1.6546279011328127, 3.142964874522785, 3.095751305731075, 0.33214104409515793, 1.7417276825556467, 0.9407898348829952, 1.2234783743498645, 2.3225110538940306, 2.0251107205830308, 1.5344166586740946, 0.6084407411965552, 2.305937330158725, 0.9865724156112016, 1.0292666803541888, 0.6808120347620694, 2.305937330158725, 3.180029593453413, 2.648360010980931, 2.794684163591902, 0.35699116052934887, 1.163136005999963, 3.406149042428964, 1.1042919666306559, 2.9509295834477345, 1.5212552126161238, 2.305937330158725, 2.3460979142677947, 2.5022319753026934, 1.4282519229408763, 2.8248467496440672, 0.9208187539523751, 0.9612061411594071, 1.870208760597288, 1.4699636289881597, 1.2086942367537974, 1.0731721660532705, 2.305937330158725, 1.9323566673461323, 0.1316071997804136, 0.6341276798040393, 1.0252957148000152, 1.1220207335910874, 0.9373968919856557, 2.5022319753026934, 1.1654864273721777, 0.9459294745354062, 2.4442400283250065, 0.9424962986970122, 2.5691787649333064, 1.6313..."
1,bsns_050.txt,"[bad, weather, hit, nestle, sale, combination, rising, raw, material, cost, sluggish, european, economy, swiss, food, drink, giant, revenue, dipped, franc, ice, cream, mineral, water, dampened, wet, summer, however, profit, margin, helped, strong, performance, america, china, raise, dividend, paying, back, debt, said, strength, dollar, disposal, business, challenging, trading, condition, europe, dented, poor, acros, continent, contrast, prolonged, heat, wave, severely, affected, demand, bottled, also, fell, although, chocolate, coffee, frozen, good, petcare, product, performed, better, elsewhere, enjoyed, exceptional, year, north, outperforming, market, term, growth, added, strongly, africa, asium, despite, impact, high, oil, price, political, instability, total, earning, interest, remained, broadly, flat, past, company, ...]","[1.764331902689513, 2.472180748071463, 1.0581327815390975, 6.959705792179311, 1.4879303031392312, 1.915966251157963, 1.4608392901444682, 2.0920575102136443, 1.496071666597875, 0.8879375275577194, 2.0685764143641214, 0.8940116752699125, 1.0441339578964615, 3.0441895121681943, 1.6845721836353762, 1.9323566673461323, 1.1351424109129926, 1.3387298435550328, 2.305937330158725, 4.055096702378891, 3.0000936346578184, 3.3425786374011723, 3.963327701360288, 2.6455648095290365, 2.745270023988988, 2.648360010980931, 1.61786500936053, 0.7271939603431928, 1.6838601168906793, 2.3793445706987026, 1.2504200023088938, 0.9856021792993573, 1.0484769389072437, 1.6990636389938378, 1.383542187971395, 1.33029667601817, 2.7541258004682, 1.6149362554939817, 0.7587006937754538, 1.3516948207194002, 0.09279990051820618, 1.6149362554939817, 1.2469594701993874, 2.5691787649333064, 1.514821102610714, 1.9323566673461323, 1.923050219097158, 1.496071666597875, 0.9493900066449127, 2.745270023988988, 1.32202415005217..."
2,bsns_057.txt,"[economy, tronger, forecast, probably, grew, faster, rate, third, quarter, reported, according, bank, england, deputy, governor, rachel, lomax, private, sector, busines, survey, suggest, stronger, official, e

<h3>The vector space model for scoring</h3>

The representation of a set of documents as vectors in a common vector space is known as the vector space model and is fundamental to a host of information retrieval operations ranging from scoring documents on a query, document classification and document clustering

In [58]:
# Save document as vector of tf-idf weights

# Format = {document: [[term in doc][tf-idf weights]]}

vecs = wtd.map(lambda d:(d[0], [d[1],d[2]]))

In [59]:
vecsMap = vecs.collectAsMap()

<h3>Computing vector scores</h3>

We have a collection of documents each represented by a vector, a free text query / other document represented by a vector, and a positive integer K. We seek the K documents of the collection with the highest vector space scores on the given query / document. 

In [60]:
# Get the norm of a document vector

def get_length(doc_name:str):
    
    s= 0
    
    for w in vecsMap[doc_name][1]:
        s += math.pow(w, 2)
    
    return math.sqrt(s)

In [61]:
def initialize_length():
    
    length = {}
    
    for doc in vecsMap:
        length[doc] = get_length(doc)
        
    return length

<h3>Cosine Score - Algorithm for computing vector space scores</h3>

The array Length holds the lengths (normalization factors) for each of the N documents, whereas the array Scores holds the scores for each of the documents. When the scores are finally computed, all that remains is to pick off the K documents with the highest scores.

The outermost loop beginning repeats the updating of Scores, iterating over each query term t in turn. Than I calculate the weight in the query vector for term t. Next, update the score of each document by adding in the contribution from term t. This process of adding in contributions one query term at a time is known as term-at-a-time scoring or accumulation, and the N elements of the array Scores are therefore known as accumulators . For this purpose, and in order to improve efficiency, I stored the document frequency DF of term t at the head of the posting group, Second, I stored the term frequency TF for each postings entry. 
Finally, extracts the top K scores.


In [62]:
# Cosine Score Algorithm

# parameters:
    # p_doc   = document name / null for a query
    # p_terms = terms of a document structured as Dict {term:num of appearances} 
    #           terms of a query structured as an array [term]
    # K       = top K closest documents 
    
def cosine_score(p_doc, p_terms, K): 
    
    N = num_of_documents
    scores = {}
    length = {}
    
    tf     = 0
    log_tf = 0
    dft    = 0
    idf    = 0
    wtd    = 0
    
    _tf     = 0
    _log_tf = 0
    _wtd    = 0
    
    inner_prod = 0
    
    length = initialize_length()
    
    for term in p_terms:
        
        if p_doc != "":
            tf = p_terms[term]
            log_tf = 1 + math.log(tf, 10)
        else:
            tf = 0
            log_tf = 0
            
        if term in inverted_indexMap:
            
            posting_list = inverted_indexMap[term] 
            
            dft = posting_list[0]
            idf = math.log(N/dft, 10)
            wtd = tf * idf            
            
            for doc in posting_list[1]:
                 
                if doc != p_doc: 
                    
                    _tf     = posting_list[1][doc]
                    _log_tf = 1 + math.log(_tf, 10)
                    _wtd    = _log_tf * idf
                    
                    if p_doc != "":
                        inner_prod = wtd * _wtd
                    else:
                        inner_prod = _wtd
                        
                    if doc in scores:
                        scores[doc] += inner_prod
                    else:
                        scores[doc] = inner_prod
                    
    for doc in scores:
             
            scores[doc] = scores[doc] / length[doc]
             
    topK = sorted(scores.items(), key = operator.itemgetter(1), reverse = True)[0:K]
    
    return topK

In [63]:
def f_topK(d):
    
    doc   = d[0]
    terms = d[1]
    
    top = cosine_score(doc, terms, K)
    
    return (doc, top)

In [64]:
documents_with_cnt = documents.map(lambda d:(d[0], dict(Counter(d[1]))))

<h3>Match top K documents</h3>

Lets find for each document related top K similar documents

In [65]:
K = 5
topK = documents_with_cnt.map(f_topK)

In [66]:
topK.take(5)

[('bsns_382.txt',
  [('pltcs_132.txt', 7.1809458633425045),
   ('pltcs_043.txt', 4.7911429032547455),
   ('pltcs_294.txt', 4.4083155656569195),
   ('pltcs_042.txt', 4.359888954236427),
   ('bsns_255.txt', 4.26921065875429)]),
 ('bsns_050.txt',
  [('bsns_111.txt', 3.893711870692363),
   ('bsns_247.txt', 3.3369617630584676),
   ('bsns_173.txt', 3.2951470011356525),
   ('bsns_201.txt', 3.1244263984803484),
   ('bsns_372.txt', 2.9898641569323248)]),
 ('bsns_057.txt',
  [('bsns_119.txt', 7.970453154660758),
   ('bsns_151.txt', 7.8523907127239525),
   ('bsns_438.txt', 7.588939406719513),
   ('bsns_290.txt', 7.568344201069349),
   ('bsns_507.txt', 6.719453813519126)]),
 ('bsns_272.txt',
  [('bsns_305.txt', 7.279201209171136),
   ('bsns_490.txt', 7.119063379073892),
   ('bsns_303.txt', 6.164799115313264),
   ('bsns_330.txt', 4.45477454202674),
   ('bsns_031.txt', 4.290688502538909)]),
 ('bsns_065.txt',
  [('bsns_075.txt', 15.150630251804516),
   ('bsns_109.txt', 5.885876659848677),
   ('bsns_4

In [67]:
similarity_example = main_data.filter(lambda d:(d[0] == "bsns_382.txt" or d[0] == "pltcs_132.txt"))

In [68]:
cols = ["Similar documents", "Text"]
se = similarity_example.toDF(cols)

<h3>Documents similarity example</h3>

bsns_382.txt and pltcs_132.txt with score of 7.3
and actually the documents are about the same subject - Ritirement age of workers

bsns_382.txt is about a new law which says that employers will no longer be able to force workers to retire before the age of 65<br>
pltcs_132.txt is about a review of how the market will be influenced by raising the retirement age <br>

In [69]:
se.toPandas()

,Similar documents,Text
0,bsns_382.txt,"Ban on forced retirement under 65\n\nEmployers will no longer be able to force workers to retire before 65, unless they can justify it.\n\nThe government has announced that firms will be barred from 2006 from imposing arbitrary retirement ages. Under new European age discrimination rules, a default retirement age of 65 will be introduced. Workers will be permitted to request staying on beyond this compulsory retirement age, although employers will have the right to refuse. Trade and Industry Secretary Patricia Hewitt said people would not be forced to work longer than they wanted, saying the default age was not a statutory, compulsory retirement age. She said employers would be free to continue employing people for as long as they were competent.\n\nUnder age discrimination proposals from the Department of Trade and Industry last year workers were to be allowed to work on till 70 if they wished.\n\nBusiness leaders had opposed the plan as they said it would be too costly and cumber..."
1,pltcs_132.txt,"Retirement age could be scrapped\n\nThe ""myth that ageing is a barrier"" to contributing to society needs to be ""exploded"", the work and pensions minister has said.\n\nThis was why the government was considering scrapping the retirement age entirely, Alan Johnson said. It was also committed to ""stamping out"" age discrimination and would outlaw it, he told a conference on ageing. All three parties have been wooing older voters with both the Tories and Lib Dems pledging higher pensions.\n\nMr Johnson told Age Concern's Age Agenda in London the government was ""seriously considering"" introducing pensions based on residency rather than national insurance contributions. This idea has been adopted by the Lib Dems as policy, while the Tories have pledged to boost pensions by restoring the link between earnings and pensions. Mr Johnson's speech comes after he last week unveiled plans to find a consensus on how to reform the country's pension system. This would be based on a series of princip..."


In [70]:
K = 1
top1 = documents_with_cnt.map(f_topK)

In [71]:
top1.take(5)

[('bsns_382.txt', [('pltcs_132.txt', 7.1809458633425045)]),
 ('bsns_050.txt', [('bsns_111.txt', 3.893711870692363)]),
 ('bsns_057.txt', [('bsns_119.txt', 7.970453154660758)]),
 ('bsns_272.txt', [('bsns_305.txt', 7.279201209171136)]),
 ('bsns_065.txt', [('bsns_075.txt', 15.150630251804516)])]

In [72]:
def f_order_key(t):
    doc = t[0]
    top = t[1][0]
    score = top[1]
    
    return -1*score

In [73]:
top_top = top1.takeOrdered(5, f_order_key)

<h3>Top documents similarity</h3>

In [74]:
top_top

[('pltcs_293.txt', [('pltcs_111.txt', 77.47816690797887)]),
 ('tech_379.txt', [('tech_009.txt', 66.5629040538705)]),
 ('tech_009.txt', [('tech_379.txt', 56.55818387168409)]),
 ('sprt_121.txt', [('sprt_112.txt', 55.20634774238921)]),
 ('sprt_112.txt', [('sprt_121.txt', 54.98052900023596)])]

In [75]:
highest_similarity = main_data.filter(lambda d:(d[0] == "pltcs_293.txt" or d[0] == "pltcs_111.txt"))
cols = ["Highest similar documents", "Text"]
hs = highest_similarity.toDF(cols)

Highest similarity in the corpus was given between documents pltcs_293.txt and pltcs_111.txt with score of 77.5
and actually, excpect of minor changes, the docuemtns are almost the same


In [76]:
hs.toPandas()

,Highest similar documents,Text
0,pltcs_293.txt,"Kilroy launches 'Veritas' party\n\nEx-BBC chat show host and East Midlands MEP Robert Kilroy-Silk said he wanted to ""change the face of British politics"" as he launched his new party. Mr Kilroy-Silk, who recently quit the UK Independence Party, said ""our country"" was being ""stolen from us"" by mass immigration. He told a London news conference that Veritas - Latin for ""truth"" - would avoid the old parties' ""lies and spin"". UKIP leader Roger Knapman says he is glad to see the back of Mr Kilroy-Silk.\n\nMr Kilroy-Silk promised a ""firm but fair"" policy on immigration and said they hoped to contest most seats at the forthcoming general election. He said Veritas would also announce detailed policies on crime, tax, pensions, health and defence over the next few weeks. Labour campaign spokesman Fraser Kemp said Veritas was joining ""an already crowded field on the right of British politics"". On Thursday Mr Kilroy-Silk is due to announce which constituency he will run in at the next general ..."
1,pltcs_111.txt,"Kilroy launches 'Veritas' party\n\nEx-BBC chat show host and East Midlands MEP Robert Kilroy-Silk has said he wants to ""change the face of British politics"" as he launched his new party.\n\nMr Kilroy-Silk, who recently quit the UK Independence Party,said ""our country"" was being ""stolen from us"" by mass immigration. He told a London news conference that Veritas - Latin for ""truth"" - would avoid the old parties' ""lies and spin"". UKIP leader Roger Knapman says he was glad to see the back of Mr Kilroy-Silk.\n\nMr Kilroy-Silk promised a ""firm but fair"" policy on immigration and said they hoped to contest most seats at the forthcoming general election. He said Veritas would also announce detailed policies on crime, tax, pensions, health and defence over the next few weeks. And he announced the party would be holding a leadership election. On Thursday he is due to announce which constituency he will run in at the next general election - that will come amid speculation he has his sights se..."


<h3>Query vector representation</h3>

Break text into individual linguistic units

In order to get precise results, we have to represent the query as a vector exactly 
as we did for document vectors, and actually I used the same functions to achieve that goal

clean and arrange the given query 

* Lower case
* Remove English stop words
* Remove English possession ('s)
* Remove symbols and numbers
* Remove empty strings
* Remove one letter words
* Singularize words

In [77]:
def query_to_vec(q):
    vec = text_to_vec(("query", q))
    vec = remove_stop_words(vec)
    vec = remove_symbols(vec)
    vec = singularize(vec)
    
    return list(dict(Counter(vec[1])))

In [78]:
# Test query to vector function

test_q_to_vec = "Show me the money! Keep your enemies close"
q_vec = query_to_vec(test_q_to_vec)
q_vec

['show', 'money', 'keep', 'enemy', 'close']

<h3>Match top K documents for a given query</h3>
<h4>Examples</h4>


As I catalog the collection of documents to different categories, 
We can check queries related to specific category and expect to get ducuments from that category 

In [79]:
K =10


# Sports
q1  = "Medal, Season, Jumpping Sport"
cs_q1 = cosine_score("",query_to_vec(q1), K)
cs_q1

[('sprt_001.txt', 0.2646074098370145),
 ('sprt_074.txt', 0.2358641978259906),
 ('sprt_039.txt', 0.21683083840242692),
 ('sprt_073.txt', 0.21643685833903073),
 ('sprt_020.txt', 0.21070485208181305),
 ('sprt_012.txt', 0.21070485208181305),
 ('sprt_028.txt', 0.20512342841706438),
 ('sprt_071.txt', 0.20295697691947373),
 ('sprt_085.txt', 0.19291667023280837),
 ('entrtmnt_189.txt', 0.18351060425646176)]

In [80]:
# Entertainment

q2  = "Best Song or a movie or a show"
cs_q2 = cosine_score("",query_to_vec(q2), K)
cs_q2

[('entrtmnt_332.txt', 0.20758210457321843),
 ('entrtmnt_139.txt', 0.20293153812552533),
 ('entrtmnt_095.txt', 0.20042568452344672),
 ('entrtmnt_172.txt', 0.19140659166400892),
 ('entrtmnt_323.txt', 0.1817630926673015),
 ('entrtmnt_235.txt', 0.1806562297166499),
 ('entrtmnt_105.txt', 0.1806562297166499),
 ('entrtmnt_264.txt', 0.18017753615465304),
 ('entrtmnt_097.txt', 0.1785207977781076),
 ('entrtmnt_195.txt', 0.17249808980171674)]

In [81]:
# Politics

q3  = "Parliament, Prime minister, Citizen"
cs_q3 = cosine_score("",query_to_vec(q3), K)
cs_q3

[('pltcs_401.txt', 0.2674146445588363),
 ('bsns_423.txt', 0.25740686783277433),
 ('pltcs_326.txt', 0.25391468952660445),
 ('pltcs_341.txt', 0.25016727048684145),
 ('pltcs_223.txt', 0.25016727048684145),
 ('pltcs_077.txt', 0.21706174156583033),
 ('pltcs_354.txt', 0.20211370854203436),
 ('bsns_106.txt', 0.186061919003983),
 ('pltcs_409.txt', 0.18438735613544305),
 ('pltcs_319.txt', 0.18270569107697013)]

In [82]:
#Business

q4  = "Stock market , my pension my money"
cs_q4 = cosine_score("",query_to_vec(q4), K)
cs_q4

[('bsns_206.txt', 0.3044557654751768),
 ('bsns_274.txt', 0.24095370863245907),
 ('bsns_267.txt', 0.23889150934670228),
 ('bsns_233.txt', 0.23751212969137966),
 ('bsns_365.txt', 0.23399301527366223),
 ('bsns_255.txt', 0.21671108708463716),
 ('bsns_238.txt', 0.21591889018789515),
 ('bsns_276.txt', 0.21054438313058688),
 ('bsns_281.txt', 0.1990690186479696),
 ('bsns_171.txt', 0.19474590869462713)]

In [83]:
# Technology

q5  = "online software website"
cs_q5 = cosine_score("",query_to_vec(q5), K)
cs_q5

[('tech_284.txt', 0.2213261110923545),
 ('tech_259.txt', 0.2203069624297773),
 ('tech_036.txt', 0.20363795173593072),
 ('tech_003.txt', 0.20363795173593072),
 ('tech_398.txt', 0.20203512100919785),
 ('tech_227.txt', 0.20203512100919785),
 ('tech_278.txt', 0.1936746689859994),
 ('tech_369.txt', 0.1807253288790651),
 ('tech_357.txt', 0.16048989996797505),
 ('tech_040.txt', 0.15797461539464341)]

Lets check what we can get when searching only one word:

In [84]:
q6  = "Retirement"
cs_q6 = cosine_score("",query_to_vec(q6), K)
cs_q6

[('bsns_255.txt', 0.18712012211101378),
 ('sprt_460.txt', 0.17298794417222538),
 ('bsns_382.txt', 0.16242504134351057),
 ('pltcs_132.txt', 0.1480887943495272),
 ('entrtmnt_269.txt', 0.1336243533540087),
 ('sprt_347.txt', 0.12919289146764054),
 ('bsns_041.txt', 0.11033156627127216),
 ('sprt_241.txt', 0.10845034038045735),
 ('sprt_462.txt', 0.1032913063926923),
 ('pltcs_357.txt', 0.09896525038310693)]

The term "retirement" can be related to pension or to a sportsman who retired from the match.

Lets look in the results :

<b>Pension</b>:

 bsns_255.txt is about the UK state pension system<br>
 bsns_382.txt is about a new law which says that employers will no longer be able to force workers to retire before the age of 65<br>
 pltcs_132.txt is about a review of how the market will be influenced by raising the retirement age <br>
            
<b>Sports</b>

 sprt_460.txt is about Lindsay Devenport- American professional tennis player who retired from the match because   of a back injury<br>
 sprt_347.txt is mention Martin Osborne Johnson - an English retired rugby player<br>

 <b>Document frequency:</b>
 
 We can also notice that the term "retirement" is a rare term (dft = 38) , so even a document with tf = 2 like sprt_347.txt will be returned in the results set and it is actually very relevant
 
 <h3>More query examples</h3>

In [85]:
q7  = "A new song"
cs_q7 = cosine_score("",query_to_vec(q7), K)
cs_q7

[('entrtmnt_160.txt', 0.17967111975677402),
 ('entrtmnt_112.txt', 0.17475636085142598),
 ('entrtmnt_159.txt', 0.13203273034832375),
 ('entrtmnt_143.txt', 0.1266481863515194),
 ('entrtmnt_123.txt', 0.1259874929662363),
 ('tech_279.txt', 0.1188480403992821),
 ('tech_302.txt', 0.1188480403992821),
 ('entrtmnt_130.txt', 0.11577645067189055),
 ('entrtmnt_127.txt', 0.11543236475089988),
 ('entrtmnt_103.txt', 0.11543236475089988)]

When searching for a new song we get the expected results:

entrtmnt_160.txt - the new singel "Band Aid" in iTunes<br>
entrtmnt_112.txt - the re-release of hit song of Elvis<br>
entrtmnt_159.txt - the new video of former Westlife

In [86]:
q8  = "Antivirus"
cs_q8 = cosine_score("",query_to_vec(q8), K)
cs_q8

[('tech_036.txt', 0.17482801114068425),
 ('tech_003.txt', 0.17482801114068425),
 ('tech_008.txt', 0.14547626288495188),
 ('tech_252.txt', 0.14547626288495188),
 ('tech_177.txt', 0.14428821106926562),
 ('tech_292.txt', 0.14428821106926562),
 ('tech_117.txt', 0.1331734366292737),
 ('tech_272.txt', 0.1322531292700624),
 ('tech_290.txt', 0.11487596186395439),
 ('tech_116.txt', 0.114778012636701)]

When quering for the term "antivirus" we gets a usefull information about viruses and tools for example :

tech_036.txt- Microsoft spyware tool<br>
tech_003.txt - Windows virus disguising itself as an electronic Christmas card<br>
tech_177.txt - Tools to clean up PCs harbouring viruses abd spyware

In [87]:
q9  = "Python"
cs_q9 = cosine_score("",query_to_vec(q9), K)
cs_q9

[('entrtmnt_290.txt', 0.17653183044330767)]

When searching for results about the language "Python" we did'nt find much - as expected in BBC news dataset.

The only document related is "entrtmnt_290.txt" with respct to "Monty Python"- British comedy troupe who created comedy tv shows

<b>Document frequency and term frequency:</b>

We should notice that although the term appears in the document only ones, it is the only document the term is exists in so the connection between the document and query is very strong, and we are getting it in the result set 


In [88]:
q10  = "Barcelona Real Madrid"
cs_q10 = cosine_score("",query_to_vec(q10), K)
cs_q10

[('sprt_151.txt', 0.36401547606179513),
 ('sprt_275.txt', 0.3603633275774207),
 ('sprt_237.txt', 0.33862164314951626),
 ('sprt_211.txt', 0.33620936445452204),
 ('sprt_183.txt', 0.3307759544486267),
 ('sprt_245.txt', 0.30721179631047085),
 ('sprt_120.txt', 0.30542369806026),
 ('sprt_238.txt', 0.2974009500997684),
 ('sprt_239.txt', 0.2933989336606566),
 ('sprt_167.txt', 0.2644620471066777)]

News and game results in the soccer spanish league
and actually all results are sports documents

<h3>Part2</h3>
<h3>K-means algorithm</h3>



In [89]:
K = 5

I will use the weights rdd vector : vecs = tuple(doc_name, [[terms], [tf-idf weights]])

vecs is the tf-idf weights vectors I used in the inverted index for Cosine Score algorithm and it is not the completed terms vector.
actually it contains ,for each document, terms and weights.
in order to run the K-means algorithm, I will transfer vecs to be a full vocabulary terms vector. 

In [90]:
vecs

PythonRDD[76] at collectAsMap at <ipython-input-59-1b8b9c4fe9e9>:1

In [91]:
vecs.take(1)

[('bsns_382.txt',
  [['ban',
    'forced',
    'retirement',
    'employer',
    'will',
    'longer',
    'able',
    'force',
    'worker',
    'retire',
    'unles',
    'can',
    'justify',
    'government',
    'announced',
    'firm',
    'barred',
    'imposing',
    'arbitrary',
    'age',
    'new',
    'european',
    'discrimination',
    'rule',
    'default',
    'introduced',
    'permitted',
    'request',
    'staying',
    'beyond',
    'compulsory',
    'although',
    'right',
    'refuse',
    'trade',
    'industry',
    'secretary',
    'patricium',
    'hewitt',
    'said',
    'person',
    'work',
    'wanted',
    'saying',
    'statutory',
    'free',
    'continue',
    'employing',
    'long',
    'competent',
    'proposal',
    'department',
    'last',
    'year',
    'allowed',
    'till',
    'wished',
    'busines',
    'leader',
    'opposed',
    'plan',
    'costly',
    'cumbersome',
    'british',
    'chamber',
    'commerce',
    'welcomed',
 

In [92]:
vocabulary.sort()
voc_ln = len(vocabulary)
voc_ln

26746

In [93]:
# convert to : tuple(doc, Dict{term index:weight})
def rebuild_vec(v):
    doc   = v[0]
    terms = v[1][0]
    wtd   = v[1][1]
    d = {}
    
    for i in range(len(terms)):
        ind = vocabulary.index(terms[i])
        d[ind] = wtd[i]
        
    return (doc, dict(sorted(d.items())))

In [94]:
rvecs = vecs.map(rebuild_vec)

In [95]:
rvecs.take(1)

[('bsns_382.txt',
  {51: 1.3605582810507055,
   54: 0.9407898348829952,
   440: 2.794684163591902,
   443: 2.8702087605972877,
   687: 1.3650587822773816,
   737: 0.9208187539523751,
   978: 1.0292666803541888,
   1214: 2.648360010980931,
   1853: 1.466516423036159,
   1952: 2.305937330158725,
   2290: 0.791027514549663,
   2343: 1.4282519229408763,
   3006: 0.8658873868146453,
   3268: 0.9290287239972048,
   3432: 0.6084407411965552,
   3852: 1.204315215062855,
   3858: 1.8032619709666744,
   4483: 2.004907334494744,
   4579: 1.9493900066449126,
   4593: 1.5691787649333067,
   4643: 2.5691787649333064,
   4662: 1.6230541457161611,
   4702: 2.8248467496440672,
   4729: 1.446487189726891,
   4983: 0.9459294745354062,
   5198: 2.116881093938676,
   5289: 3.3473300153169503,
   5541: 2.8702087605972877,
   5874: 1.1493780705368775,
   5929: 2.9509295834477345,
   5961: 2.305937330158725,
   6108: 1.3261407162470122,
   6442: 0.835446654338076,
   6527: 3.406149042428964,
   7523: 1.684572

In [97]:
import numpy as np

# v  = tuple (doc, Dict{term index:weight})
def np_vec(v):
    doc = v[0]
    wtd = v[1]
    arr = []
    
    for i in range(voc_ln):
        if i in wtd:
            arr.append(wtd[i])
        else:
            arr.append(float(0))
    
    return(doc, np.array(arr))

In [98]:
npvecs = rvecs.map(np_vec)
npvecs.take(1)

[('bsns_382.txt', array([0., 0., 0., ..., 0., 0., 0.]))]

<b>K-means</b> 

Is the most important flat clustering algorithm. Its objective is to minimize the average squared Euclidean distance of documents from their cluster centers.

A measure of how well the centroids represent the members of their clusters is the residual sum of squares or RSS , the squared distance of each vector from its centroid summed over all vectors

RSS is the objective function in K-means and our goal is to minimize it. Since N is fixed, minimizing RSS is equivalent to minimizing the average squared distance, a measure of how well centroids represent their documents. 

The first step of K-means is to select as initial cluster centers K randomly selected documents, the seeds . 
The algorithm then moves the cluster centers around in space in order to minimize residual sum of squares. 
this is done iteratively by repeating two steps until a stopping criterion is met: reassigning documents to the cluster with the closest centroid; and recomputing each centroid based on the current members of its cluster.

In [212]:
# Select as initial cluster centers ,K randomly selected documents - the seeds

#seeds = npvecs.map(lambda x:(x[1])).take(5)

<b>Seeds Selection</b>

When I used randonally seeds, the algorithm was stopped after few iterations when there was no change between centoids and new centroids.I got only 2 main categories with no significant results .
Instead, I choose the seeds to be K documents contains terms with strong category relation.  

In [230]:
seeds = npvecs.filter(lambda x:(x[0] == cs_q1[0][0] or x[0] == cs_q2[0][0] or x[0] == cs_q3[0][0] or 
                               x[0] == cs_q4[0][0] or x[0] == cs_q5[0][0]))

In [231]:
seeds.take(5)

[('bsns_206.txt', array([0., 0., 0., ..., 0., 0., 0.])),
 ('entrtmnt_332.txt', array([0., 0., 0., ..., 0., 0., 0.])),
 ('pltcs_401.txt', array([0., 0., 0., ..., 0., 0., 0.])),
 ('sprt_001.txt',
  array([2.17123876, 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ])),
 ('tech_284.txt', array([0., 0., 0., ..., 0., 0., 0.]))]

In [234]:
seeds = seeds.map(lambda x:(x[1])).take(5)

In [235]:
seeds

[array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([2.17123876, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]),
 array([0., 0., 0., ..., 0., 0., 0.])]

In [236]:
centroids = seeds


Step 1 - Reassigning documents to the cluster with the closest centroid


In [270]:
# d = (doc, vec)
# return value : (v1, 1, doc)
#               v1 = document vector, 
#               1 = Count for later use (centroids vector avarage)
#               doc = document index(name)

def get_closest_centroid(d, centroids):
    
    doc = d[0]
    v1 = d[1]
    
    distance     = 0 
    min_distance = 0
    centroid = 0
    
    min_distance = np.linalg.norm(v1 - centroids[0])

    for i in range(1, K):
        
        distance = np.linalg.norm(v1 - centroids[i])
        
        if min_distance > distance:
            min_distance = distance
            centroid     = i
    
    return(centroid, (v1, 1, doc))

In [271]:
closest_centroids = npvecs.map(lambda v: get_closest_centroid(v, centroids)) 

In [272]:
closest_centroids.take(10)

[(4, (array([0., 0., 0., ..., 0., 0., 0.]), 1, 'bsns_382.txt')),
 (4, (array([0., 0., 0., ..., 0., 0., 0.]), 1, 'bsns_050.txt')),
 (4, (array([0., 0., 0., ..., 0., 0., 0.]), 1, 'bsns_057.txt')),
 (2, (array([0., 0., 0., ..., 0., 0., 0.]), 1, 'bsns_272.txt')),
 (4, (array([0., 0., 0., ..., 0., 0., 0.]), 1, 'bsns_065.txt')),
 (4, (array([0., 0., 0., ..., 0., 0., 0.]), 1, 'bsns_002.txt')),
 (4, (array([0., 0., 0., ..., 0., 0., 0.]), 1, 'bsns_064.txt')),
 (4, (array([0., 0., 0., ..., 0., 0., 0.]), 1, 'bsns_491.txt')),
 (4, (array([0., 0., 0., ..., 0., 0., 0.]), 1, 'bsns_110.txt')),
 (4, (array([0., 0., 0., ..., 0., 0., 0.]), 1, 'bsns_036.txt'))]

In [273]:
# tuple (sum,count)
def sum_vecs(v1,v2):
    
    return (v1[0] + v2[0], v1[1] + v2[1])

In [274]:
sum_closest_centroids = closest_centroids.reduceByKey(sum_vecs)

In [275]:
new_centroids = sum_closest_centroids.map(lambda x:(x[1][0] / x[1][1]))


Step 2 - Recomputing each centroid based on the current members of its cluster.


In [276]:
new_centroids.take(5)

[array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0.37601571, 0.05695892, 0.        , ..., 0.        , 0.09551968,
        0.        ]),
 array([0.00357896, 0.00167379, 0.00271671, ..., 0.00183919, 0.        ,
        0.00334758])]

In [277]:
centroids = new_centroids.take(5)

Move the cluster centers around in space in order to minimize residual sum of squares RSS
iteratively by repeating two steps until a stopping criterion is met

Apply one of the following termination conditions.

* A fixed number of iterations has been completed. This condition limits the runtime of the clustering algorithm, but in some cases the quality of the clustering will be poor because of an insufficient number of iterations.
* Centroids do not change between iterations.

In [278]:
num_of_itr = 20
loop = True

while num_of_itr > 0 and loop == True:
    loop = False
    
    closest_centroids = npvecs.map(lambda v: get_closest_centroid(v, centroids)) 
    sum_closest_centroids = closest_centroids.reduceByKey(sum_vecs)
    
    prnt = sum_closest_centroids.take(K)
    for i in range(K):
        print(prnt[i][0], prnt[i][1][1])
    
    print("--------")
    
    new_centroids = sum_closest_centroids.map(lambda x:(x[1][0] / x[1][1])).take(K) 
    
    for i in range(K):
        if not np.array_equal(centroids[i], new_centroids[i]):
            loop = True
    
    centroids = new_centroids
    
    num_of_itr = num_of_itr - 1

0 2
1 40
2 314
3 92
4 1777
--------
0 2
1 40
2 314
3 133
4 1736
--------
0 2
1 40
2 313
3 185
4 1685
--------
0 2
1 40
2 313
3 204
4 1666
--------
0 2
1 40
2 313
3 216
4 1654
--------
0 2
1 40
2 313
3 221
4 1649
--------
0 2
1 40
2 313
3 232
4 1638
--------
0 2
1 40
2 313
3 243
4 1627
--------
0 2
1 40
2 314
3 256
4 1613
--------
0 2
1 40
2 314
3 267
4 1602
--------
0 2
1 40
2 314
3 275
4 1594
--------
0 2
1 40
2 315
3 285
4 1583
--------
0 2
1 40
2 315
3 297
4 1571
--------
0 2
1 40
2 315
3 312
4 1556
--------
0 2
1 40
2 315
3 333
4 1535
--------
0 2
1 40
2 314
3 350
4 1519
--------
0 2
1 40
2 314
3 367
4 1502
--------
0 2
1 40
2 314
3 379
4 1490
--------
0 2
1 40
2 314
3 393
4 1476
--------
0 2
1 40
2 314
3 406
4 1463
--------


And after 5 more iterations ..


0: 2

1: 41

2: 314

3: 445

4: 1423

<b>K-means results :</b>


In [280]:
test_results = closest_centroids.map(lambda x:(x[0],[x[1][2]])).reduceByKey(lambda x,y:x+y)

In [283]:
cols = ["centroid", "docs"]
DF = test_results.toDF(cols)
DF.toPandas()

,centroid,docs
0,0,"[bsns_281.txt, bsns_206.txt]"
1,1,"[entrtmnt_064.txt, entrtmnt_081.txt, entrtmnt_377.txt, entrtmnt_095.txt, entrtmnt_328.txt, entrtmnt_034.txt, entrtmnt_367.txt, entrtmnt_326.txt, entrtmnt_084.txt, entrtmnt_264.txt, entrtmnt_253.txt, entrtmnt_363.txt, entrtmnt_201.txt, entrtmnt_097.txt, entrtmnt_325.txt, entrtmnt_078.txt, entrtmnt_357.txt, entrtmnt_069.txt, entrtmnt_082.txt, entrtmnt_092.txt, entrtmnt_368.txt, entrtmnt_035.txt, entrtmnt_063.txt, entrtmnt_314.txt, entrtmnt_372.txt, entrtmnt_062.txt, entrtmnt_275.txt, entrtmnt_038.txt, entrtmnt_091.txt, entrtmnt_039.txt, entrtmnt_354.txt, entrtmnt_051.txt, entrtmnt_353.txt, entrtmnt_030.txt, entrtmnt_370.txt, entrtmnt_075.txt, entrtmnt_352.txt, entrtmnt_323.txt, entrtmnt_373.txt, entrtmnt_086.txt, entrtmnt_355.txt]"
2,2,"[bsns_245.txt, bsns_241.txt, bsns_277.txt, bsns_487.txt, bsns_146.txt, bsns_284.txt, bsns_249.txt, bsns_106.txt, bsns_504.txt, bsns_287.txt, bsns_154.txt, bsns_159.txt, entrtmnt_131.txt, entrtmnt_256.txt, pltcs_382.txt, pltcs_050.txt, pltcs_272.txt, pltcs_065.txt, pltcs_002.txt, pltcs_110.txt, pltcs_036.txt, pltcs_099.txt, pltcs_285.txt, pltcs_402.txt, pltcs_246.txt, pltcs_322.txt, pltcs_027.txt, pltcs_362.txt, pltcs_270.txt, pltcs_399.txt, pltcs_073.txt, pltcs_116.txt, pltcs_245.txt, pltcs_339.txt, pltcs_081.txt, pltcs_184.txt, pltcs_315.txt, pltcs_411.txt, pltcs_303.txt, pltcs_297.txt, pltcs_409.txt, pltcs_383.txt, pltcs_143.txt, pltcs_165.txt, pltcs_194.txt, pltcs_095.txt, pltcs_398.txt, pltcs_217.txt, pltcs_241.txt, pltcs_200.txt, pltcs_342.txt, pltcs_112.txt, pltcs_083.txt, pltcs_104.txt, pltcs_328.txt, pltcs_319.txt, pltcs_238.txt, pltcs_022.txt, pltcs_244.txt, pltcs_266.txt, pltcs_345.txt, pltcs_347.txt, pltcs_067.txt, pltcs_118.txt, pltcs_290.txt, pltcs_034.txt, pltcs_410.t..."
3,3,"[sprt_382.txt, sprt_057.txt, sprt_272.txt, sprt_065.txt, sprt_002.txt, sprt_064.txt, sprt_491.txt, sprt_110.txt, sprt_099.txt, sprt_285.txt, sprt_458.txt, sprt_402.txt, sprt_246.txt, sprt_322.txt, sprt_362.txt, sprt_418.txt, sprt_270.txt, sprt_399.txt, sprt_073.txt, sprt_245.txt, sprt_433.txt, sprt_437.txt, sprt_339.txt, sprt_483.txt, sprt_184.txt, sprt_195.txt, sprt_315.txt, sprt_411.txt, sprt_303.txt, sprt_463.txt, sprt_297.txt, sprt_509.txt, sprt_409.txt, sprt_123.txt, sprt_377.txt, sprt_248.txt, sprt_381.txt, sprt_383.txt, sprt_286.txt, sprt_165.txt, sprt_194.txt, sprt_095.txt, sprt_398.txt, sprt_478.txt, sprt_114.txt, sprt_488.txt, sprt_392.txt, sprt_484.txt, sprt_217.txt, sprt_241.txt, sprt_200.txt, sprt_342.txt, sprt_138.txt, sprt_187.txt, sprt_428.txt, sprt_083.txt, sprt_178.txt, sprt_481.txt, sprt_104.txt, sprt_328.txt, sprt_462.txt, sprt_308.txt, sprt_319.txt, sprt_238.txt, sprt_435.txt, sprt_022.txt, sprt_150.txt, sprt_185.txt, sprt_489.txt, sprt_244.txt, sprt_266.txt, s..."
4,4,"[bsns_382.txt, bsns_050.txt, bsns_057.txt, bsns_272.txt, bsns_065.txt, bsns_002.txt, bsns_064.txt, bsns_491.txt, bsns_110.txt, bsns_036.txt, bsns_099.txt, bsns_285.txt, bsns_131.txt, bsns_458.txt, bsns_402.txt, bsns_246.txt, bsns_322.txt, bsns_027.txt, bsns_362.txt, bsns_418.txt, bsns_270.txt, bsns_399.txt, bsns_073.txt, bsns_116.txt, bsns_433.txt, bsns_437.txt, bsns_339.txt, bsns_098.txt, bsns_016.txt, bsns_483.txt, bsns_081.txt, bsns_184.txt, bsns_195.txt, bsns_017.txt, bsns_315.txt, bsns_411.txt, bsns_303.txt, bsns_463.txt, bsns_297.txt, bsns_509.txt, bsns_409.txt, bsns_123.txt, bsns_377.txt, bsns_248.txt, bsns_381.txt, bsns_383.txt, bsns_286.txt, bsns_143.txt, bsns_023.txt, bsns_165.txt, bsns_194.txt, bsns_095.txt, bsns_398.txt, bsns_478.txt, bsns_114.txt, bsns_488.txt, bsns_392.txt, bsns_484.txt, bsns_217.txt, bsns_200.txt, bsns_342.txt, bsns_112.txt, bsns_138.txt, bsns_187.txt, bsns_428.txt, bsns_083.txt, bsns_178.txt, bsns_481.txt, bsns_104.txt, bsns_328.txt, bsns_462.txt, b..."


Results:

<b>Centroid 0:</b>

We can see that centroid 0 contains only 2 documents. 
Documents that are far from any other documents and therefore do not fit well into any cluster. Frequently, if an outlier is chosen as an initial seed, like in this case, then no other vector is assigned to it during subsequent iterations.

<b>Centroid 1:</b>

Contains only entertainment ducuments

<b>Centroid 2:</b>

Except of few politics documents, contains only business ducuments

<b>Centroid 3:</b>

Contains only sport ducuments

<b>Centroid 4:</b>

Contains all tech documents and more documents from others categories (Inconclusive)

In [286]:
# Centroid 0 documents

res = main_data.filter(lambda d:(d[0] == "bsns_281.txt" or d[0] == "bsns_206.txt"))
cols = ["Centroid 0 documents", "Text"]
res = res.toDF(cols)
res.toPandas()

,Centroid 0 documents,Text
0,bsns_281.txt,"Axa Sun Life cuts bonus payments\n\nLife insurer Axa Sun Life has lowered annual bonus payouts for up to 50,000 with-profits investors.\n\nRegular annual bonus rates on former Axa Equity & Law with-profits policies are to be cut from 2% to 1% for 2004. Axa blamed a poor stock market performance for the cut, adding that recent gains have not yet offset the market falls seen in 2001 and 2002. The cut will hit an estimated 3% of Axa's policyholders. The rest will know their fate in March.\n\nThe cuts on Axa's policies will mean a policyholder who had invested £50 a month into an endowment policy for the past 25 years would see a final maturity payout of £46,998. This equated to a annual investment growth rate of 8% Axa said. With-profits policies are designed to smooth out the peaks and troughs of stock market volatility. However, heavy stock market falls throughout 2001 and 2002 forced most firms to trim bonus rates on their policies. ""The stock market has grown over the past 18 mont..."
1,bsns_206.txt,"Standard Life cuts policy bonuses\n\nStandard Life, Europe's largest mutual life insurer, has cut bonuses for with-profit policyholders.\n\nAnnual bonus rates on its with-profits life policies were cut from 2.5% to 2%, while bonuses on pension policies were reduced from 3.25% to 2.5%. It is the sixth time in three years Standard Life has made cuts to bonus rates, despite an 8.7% rise in the value of the with-profits fund in 2004. The insurer blamed the cuts on poor share returns and low interest rates.\n\nWith-profits policies are designed to smooth out the peaks and troughs of stock market volatility. Profits made in good years are kept in reserve to pay investors an annual bonus even when the stock market performs badly. Slumping share prices throughout 2001 and 2002 forced most firms to trim bonus rates on their policies. Standard Life came in for criticism for sticking with stock market investments during 2001 and 2002. The insurer argued that shares outperformed other investme..."


<b>K-means Results and Time complexity</b>

The convergence of K-means is unfortunately no guarantee that a global minimum in the objective function will be reached. This is a particular problem if a document set contains many outliers , documents that are far from any other documents and therefore do not fit well into any cluster. Frequently, if an outlier is chosen as an initial seed, then no other vector is assigned to it during subsequent iterations. Thus, we end up with a singleton cluster (a cluster with only one document) even though there is probably a clustering with lower RSS
Another type of suboptimal clustering that frequently occurs is one with empty clusters. 

What is the time complexity of K-means? 

Most of the time is spent on computing vector distances. One such operation costs Theta(M). The reassignment step computes KN distances, so its overall complexity is Theta(KNM). 
In the recomputation step, each vector gets added to a centroid once, so the complexity of this step is Theta(NM). For a fixed number of iterations I, the overall complexity is therefore Theta(IKNM). Thus, K-means is linear in all relevant factors: iterations, number of clusters, number of vectors and dimensionality of the space.


<h2>The End !!!! </h2>

<b>Atempt to work with "CombineByKey" function:</b>

During the development, I have tried to use the "CombineByKey" RDD function instead of "ReduceByKey" in order to calculates the new centroids .

Unfortunately, It did not end well as I got multiple vectors in the centroids array.
Here is the code and explanations I added during the development <u>but it is not a part of the final solution.</u>

Spark combineByKey is a generic function to combine the elements for each key using a custom set of aggregation functions

Internally spark combineByKey function efficiently combines the values of a PairRDD partition by applying aggregation function. The main objective of combineByKey transformation is transforming any PairRDD[(K,V)] to the RDD[(K,C)] where C is the result of any aggregation of all values under key K.

Spark combineByKey function uses following three functions as an argument,

    createCombiner
    mergeValue
    mergeCombiners
    
CreateCombiner function of combineByKey

    This function is a first argument of combineByKey function
    It is a first aggregation step for each key
    It will be executed when any new key is found in a partition
    Execution of this lambda function is local to a partition of a node, on each individual values
    


In [ ]:
# Do not use

# tpl = (vec,doc)

def createCombiner(tpl):
    return (tpl[0], 1)

MergeValue function of combineByKey

    Second function executes when next subsequent value is given to combiner
    It also executes locally on each partition of a node and combines all values
    Arguments of this function are a accumulator and a new value
    It combines a new value in existing accumulator

In [266]:
# Do not use

# accumulator = (sum,count)
# element = (vec,doc)

def mergeValue(accumulator, element): 
    return (accumulator[0] + element[0], accumulator[1] + 1)

MergeCombiner function of combineByKey

    Final function is used to combine how to merge two accumulators (i.e. combiners) of a single key across the partitions to generate final expected result
    Arguments are two accumulators (i.e. combiners)
    Merge results of a single key from different partitions

In [267]:
# Do not use

# accumulators = (sum,count)

def mergeCombiner(accumulator1, accumulator2): 
    return (accumulator1[0], accumulator2[0], accumulator1[1] + accumulator2[1])

In [268]:
# Do not use
sum_closest_centroids = closest_centroids.combineByKey(createCombiner, mergeValue, mergeCombiner)

In [269]:
# Do not use - Unexpected results
sum_closest_centroids.take(10)

[(0, (array([0., 0., 0., ..., 0., 0., 0.]), 2)),
 (1, (array([0., 0., 0., ..., 0., 0., 0.]), 36)),
 (2,
  (array([0., 0., 0., ..., 0., 0., 0.]),
   array([0., 0., 0., ..., 0., 0., 0.]),
   array([5., 5., 5., ..., 5., 5., 5.]))),
 (3,
  (array([0., 0., 0., ..., 0., 0., 0.]),
   array([29.70524121,  4.49975451,  0.        , ...,  0.        ,
           7.54605453,  0.        ]),
   array([75., 75., 75., ..., 75., 75., 75.]))),
 (4,
  (array([0., 0., 0., ..., 0., 0., 0.]),
   array([4.34247751, 3.04630002, 4.94441231, ..., 0.        , 0.        ,
          0.        ]),
   array([396., 396., 396., ..., 396., 396., 396.])))]